# Practical test of standardiser coverage on IEDB data

## Setup

In [1]:
import sys
import os
from pathlib import Path

if not 'PROJECT_PATH' in globals():
    PROJECT_PATH = Path.cwd().parent.resolve()

sys.path.append(PROJECT_PATH)
os.chdir(PROJECT_PATH)

In [2]:
import gzip
from itertools import product
import json
import pandas as pd
import re
import tidytcells

In [3]:
with gzip.open(Path('data')/'iedb_species_decoder.json.gz', 'r') as f:
    species_decoder = json.load(f)

In [4]:
df = pd.read_csv(Path('data')/'iedb.csv.zip')
df = df[df['Response Type'] == 'T cell']

/tmp/ipykernel_7333/2744186113.py:1: DtypeWarning: Columns (8,10,12,14,16,17,18,19,20,21,22,23,24,25,26,31,32,37,38,45,46,47,48,49,50,51,52,53,54,60,61,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Path('data')/'iedb.csv.zip')


In [5]:
df.head()

,Group Receptor ID,Receptor ID,Reference IRI,Epitope IRI,Description,Antigen,Organism,Response Type,Assay IDs,MHC Allele Names,...,Chain 2 CDR1 Start Curated,Chain 2 CDR1 End Curated,Chain 2 CDR1 Start Calculated,Chain 2 CDR1 End Calculated,Chain 2 CDR2 Curated,Chain 2 CDR2 Calculated,Chain 2 CDR2 Start Curated,Chain 2 CDR2 End Curated,Chain 2 CDR2 Start Calculated,Chain 2 CDR2 End Calculated
145,47,57,http://www.iedb.org/reference/1004539,http://www.iedb.org/epitope/69921,VMAPRTLIL,"HLA class I histocompatibility antigen, Cw-3 a...",Homo sapiens (human),T cell,"1548960, 1583178","HLA-E*01:01, HLA-E*01:03",...,NaN,NaN,25.0,29.0,NaN,FVKESK,NaN,NaN,47.0,52.0
146,47,57,http://www.iedb.org/reference/1004539,http://www.iedb.org/epitope/69921,VMAPRTLIL,"HLA class I histocompatibility antigen, Cw-3 a...",Homo sapiens (human),T cell,1583178,HLA-E*01:03,...,NaN,NaN,25.0,29.0,NaN,FVKESK,NaN,NaN,47.0,52.0
147,8493,58,http://www.iedb.org/reference/1004580,http://www.iedb.org/epitope/16878,FLRGRAYGL,nuclear antigen EBNA-3,Human herpesvirus 4 (Epstein Barr virus),T cell,"1814845, 1814846, 1814847",HLA-B8,...,NaN,NaN,25.0,29.0,NaN,FQNEAQ,NaN,NaN,47.0,52.0
148,8493,58,http://www.iedb.org/reference/1004580,http://www.iedb.org/epitope/144889,FLRGRFYGL,NaN,NaN,T cell,1831737,HLA-B8,...,NaN,NaN,25.0,29.0,NaN,FQNEAQ,NaN,NaN,47.0,52.0
149,8493,58,http://www.iedb.org/reference/1017865,http://www.iedb.org/epitope/142137,EEYLQAFTY,ATP-binding cassette sub-family D member 3,Homo sapiens (human),T cell,1778798,HLA-B*44:05,...,NaN,NaN,25.0,29.0,NaN,FQNEAQ,NaN,NaN,47.0,52.0


In [6]:
def rename_species(species):
    if type(species) != str:
        return pd.NA

    if 'sapiens' in species.lower():
        return 'HomoSapiens'
    
    if 'musculus' in species.lower():
        return 'MusMusculus'
    
    return species

df['Chain 1 Species'] = df['Chain 1 Species'].map(lambda x: pd.NA if pd.isna(x) else species_decoder[str(int(x))])
df['Chain 1 Species'] = df['Chain 1 Species'].map(rename_species)

In [7]:
tcr_genes = pd.concat(
    [
        df[[' '.join(parts), 'Chain 1 Species']].rename(columns={' '.join(parts): 'gene', 'Chain 1 Species': 'species'}) for parts in product(
            ('Curated', 'Calculated'),
            ('Chain 1', 'Chain 2'),
            ('V Gene', 'J Gene')
        )
    ]
)
tcr_genes = tcr_genes[tcr_genes['gene'].notna()]

In [8]:
tcr_genes

,gene,species
145,TRAV26-1*01,HomoSapiens
415,TRAV21*01,HomoSapiens
896,TRAV26-2*01,HomoSapiens
897,TRAV26-2*01,HomoSapiens
1524,TCRAV1-2,HomoSapiens
...,...,...
209499,TRBJ1-2*01,HomoSapiens
209500,TRBJ2-7*01,HomoSapiens
209501,TRBJ1-1*01,HomoSapiens
209502,TRBJ2-3*01,HomoSapiens


In [9]:
mhc_genes = df[['MHC Allele Names', 'Chain 1 Species']].rename(columns={'MHC Allele Names': 'gene', 'Chain 1 Species':'species'})
mhc_genes = mhc_genes[mhc_genes['gene'].notna()]
mhc_genes['gene'] = mhc_genes['gene'].str.split(', ')
mhc_genes = mhc_genes.explode('gene')

In [10]:
mhc_genes

,gene,species
145,HLA-E*01:01,HomoSapiens
145,HLA-E*01:03,HomoSapiens
146,HLA-E*01:03,HomoSapiens
147,HLA-B8,HomoSapiens
148,HLA-B8,HomoSapiens
...,...,...
211479,HLA-DRB1*15:01,<NA>
211480,HLA-DRB1*15:01,<NA>
211481,HLA-DRB1*15:01,<NA>
211482,HLA-DRB1*15:01,<NA>


## Test TCR standardisation

In [11]:
tcr_genes.apply(
    lambda row: pd.NA if row.isna()['gene'] else tidytcells.tcr.standardise(row['gene'], species=row['species']),
    axis=1
).notna().sum() / len(tcr_genes)

/home/yutanagano/Projects/tidytcells/src/tidytcells/tcr/_main.py:88: UserWarning: Failed to standardise: "TCRAV1-2" for species HomoSapiens. Attempted fix "TCRAV1-2" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tidytcells/src/tidytcells/tcr/_main.py:88: UserWarning: Failed to standardise: "TCRAV26-1" for species HomoSapiens. Attempted fix "TCRAV26-1" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tidytcells/src/tidytcells/tcr/_main.py:88: UserWarning: Failed to standardise: "TCRAV21" for species HomoSapiens. Attempted fix "TCRAV21" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tidytcells/src/tidytcells/tcr/_main.py:88: UserWarning: Failed to standardise: "TCRAV8-3*01" for species HomoSapiens. Attempted fix "TCRAV8-3*01" did not meet the standardised format requirements. Ignoring this gene na

0.9472613689293347

## Test MHC standardisation

In [12]:
mhc_genes.apply(
    lambda row: pd.NA if pd.isna(row['gene']) else tidytcells.mhc.standardise(row['gene'], species=row['species']),
    axis=1
).notna().sum() / len(mhc_genes)

/home/yutanagano/Projects/tidytcells/src/tidytcells/mhc/_main.py:145: UserWarning: Failed to standardise: "E63Q mutant" for species HomoSapiens. Attempted fix "HLA-E63QMUTANT" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tidytcells/src/tidytcells/mhc/_main.py:145: UserWarning: Failed to standardise: "human CD1d" for species HomoSapiens. Attempted fix "HLA-HUMANCD1D" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tidytcells/src/tidytcells/mhc/_main.py:145: UserWarning: Failed to standardise: "human MR1" for species HomoSapiens. Attempted fix "HLA-HUMANMR*01" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tidytcells/src/tidytcells/mhc/_main.py:208: UserWarning: Unsupported species: "MusMusculus". Skipping MHC gene standardisation procedure...
  warn(
/home/yutanagano/Projects/tidytcells/src/tid

0.9976352335631737